In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [95]:
batch_size = 5  # Batch size for training.
epochs = 400  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "seq2seq.txt"

In [96]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    
    lines = f.read().split("\n")
for line in lines[1:141]:
    input_text, target_text = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 140
Number of unique input tokens: 75
Number of unique output tokens: 62
Max sequence length for inputs: 204
Max sequence length for outputs: 68


In [97]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/400
23/23 [==============================] - 18s 601ms/step - loss: 2.0314 - accuracy: 0.5743 - val_loss: 1.6415 - val_accuracy: 0.5956
Epoch 2/400
23/23 [==============================] - 9s 374ms/step - loss: 1.5501 - accuracy: 0.6099 - val_loss: 1.5793 - val_accuracy: 0.6014
Epoch 3/400
23/23 [==============================] - 8s 354ms/step - loss: 1.5040 - accuracy: 0.6132 - val_loss: 1.5264 - val_accuracy: 0.5998
Epoch 4/400
23/23 [==============================] - 8s 365ms/step - loss: 1.4530 - accuracy: 0.6148 - val_loss: 1.9284 - val_accuracy: 0.5898
Epoch 5/400
23/23 [==============================] - 8s 349ms/step - loss: 1.4002 - accuracy: 0.6292 - val_loss: 1.4421 - val_accuracy: 0.6029
Epoch 6/400
23/23 [==============================] - 8s 341ms/step - loss: 1.3380 - accuracy: 0.6494 - val_loss: 1.4267 - val_accuracy: 0.6103
Epoch 7/400
23/23 [==============================] - 8s 354ms/step - loss: 1.2142 - accuracy: 0.6836 - val_loss: 1.3533 - val_accuracy: 0.624

In [60]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [61]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    
    
input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Abbotsford Regional Hospital and Cancer Centre Fraser Health Authority Suite 400 Central City Tower 13450 - 102 Avenue Surrey BC  V3T 0H1
Decoded sentence: Abbotsford Regional Hospital and Cancer Centre

-
Input sentence: Cormorant Island Community Health Centre Vancouver Island Health Authority 1952 Bay Street Victoria, BC, V8R 1J8
Decoded sentence: Cormorant Island Community Health Centre

-
Input sentence: Ashcroft and District General Hospital Interior Health Authority Community Health and Services Centre 5th Floor 505 Doyle Avenue Kelowna, BC, V1Y 6V8
Decoded sentence: Ashcroft and District General Hospital

-
Input sentence: R.W. Large Memorial Hospital Vancouver Coastal Health Authority 11th Floor – 601 W. Broadway Vancouver BC  V6Z 1Y6
Decoded sentence: R.W. Large Memorial Hospital

-
Input sentence: Bella Coola General Hospital Vancouver Coastal Health Authority 11th Floor – 601 W. Broadway Vancouver BC  V6Z 1Y6
Decoded sentence: Bella Coola General Hospital


-
Input sentence: Arrow Lakes Hospital Interior Health Authority Community Health and Services Centre 5th Floor 505 Doyle Avenue Kelowna, BC, V1Y 6V8
Decoded sentence: Arrow Lakes Hospital

-
Input sentence: Nanaimo Regional General Hospital Vancouver Island Health Authority 1952 Bay Street Victoria BC  V8R 1J8
Decoded sentence: Nanaimo Regional General Hospital

-
Input sentence: Kootenay Lake Hospital Interior Health Authority Community Health and Services Centre 5th Floor 505 Doyle Avenue Kelowna, BC, V1Y 6V8
Decoded sentence: South Okanagan General Hospital

-
Input sentence: Slocan Community Health Centre Interior Health Authority Community Health and Services Centre 5th Floor 505 Doyle Avenue Kelowna, BC, V1Y 6V8
Decoded sentence: Slocan Community Health Centre

-
Input sentence: Royal Columbian Hospital Fraser Health Authority Suite 400 Central City Tower 13450 - 102 Avenue Surrey BC  V3T 0H1
Decoded sentence: Royal Columbian Hospital

-
Input sentence: Lions Gate Hospital Vanco

In [70]:
encoder_input_data[1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [93]:
new_input = "North York General Hospital Fraser Health Authority, 194 King Street East Toronto ON V1K 4H5"

new_encoder_input_data = np.zeros(
    (1,max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

for t, char in enumerate(new_input):
    new_encoder_input_data[0,t, input_token_index[char]] = 1.0

In [94]:
decode_sequence(new_encoder_input_data)

'South Okanagan General Hospital\n'

88